In [ ]:
from pytreenet.time_evolution.tdvp_algorithms.TJM import normalize_ttn
from pytreenet.time_evolution.tdvp_algorithms import GSE_TJM_TDVP , GSE_TJM_TDVP_random

In [ ]:
import pytreenet as ptn
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

def Visualize_tree(tree_dict, lattice_dim_squred):
    """
    Visualize the tree structure using the dictionary
    """
    G_tree = nx.Graph()  # Use an undirected graph
    for node, data in tree_dict.items():
        if data['parent'] is not None:
            G_tree.add_edge(data['parent'], node)
        for child in data['children']:
            G_tree.add_edge(node, child)

    # Define positions based on the 4x4 lattice
    lattice_nodes = [(i, j) for i in range(lattice_dim_squred) for j in range(lattice_dim_squred)]
    pos_tree = {node: (node[1], -node[0]) for node in lattice_nodes}

    # Draw the graph
    plt.figure(figsize=(3, 3))
    nx.draw(
        G_tree,
        pos_tree,
        with_labels=False,  # Do not show labels on the vertices
        node_size=50,      # Smaller circles
        node_color="Red", # Black circles
        edge_color="red",
        linewidths=1,
        width=1
    )
    plt.show()

    return

In [ ]:
import pickle
# Load the Pickle file
with open('tdvp_profiling_data_3.pkl', 'rb') as f:
    profiling_data = pickle.load(f)

# Extract profiling details from the loaded data
profiling_details = profiling_data['profiling_details']

config_id = 'Config 32'
bond_dim = 7
ttn = profiling_details[config_id]['Spanning_tree']
print(ttn.root_id)
ptn.Visualize_tree(profiling_details[config_id]['tree_dict'],4)

In [ ]:
# Load the Pickle file
with open('tdvp_profiling_data_3x3.pkl', 'rb') as f:
    profiling_data = pickle.load(f)

# Extract profiling details from the loaded data
profiling_details = profiling_data['profiling_details']

config_id = 'Config 13'
bond_dim = 3
ttn = profiling_details[config_id]['Spanning_tree']
Visualize_tree(profiling_details[config_id]['tree_dict'],3)  

In [ ]:
def get_ttn_list(profiling_details, numbers):
    ttn_list = []
    ttno_list = []
    for n in numbers:
        config_id = f'Config {n}'
        if config_id in profiling_details and 'Spanning_tree' in profiling_details[config_id]:
            ttn = profiling_details[config_id]['Spanning_tree']
            # initialize the TTN
            black_state = np.array([-1+2j , 1j])
            white_state = np.array([1 , -2j])
            ttn = ptn.alternating_product_state(ttn, black_state, white_state, bond_dim,  pattern = "checkerboard")
            local_tensor = np.array([0 , 1])
            ttn = ptn.uniform_product_state(ttn,local_state=local_tensor, bond_dim = bond_dim)
            # add the TTN to the list
            ttn_list.append(ttn)

            # define ttno
            Lx = 3
            Ly = 3
            J_z =  1 
            h_x = 0.5
            H1 = ptn.Transverse_Field_Ising(J_z, h_x, Lx, Ly )
            H1 = H1.pad_with_identities(ttn , symbolic= True)
            ttno = ptn.TTNO.from_hamiltonian(H1, ttn)
            ttno_list.append(ttno)
        else:
            ttn_list.append(None)  # Append None if the key is missing
    return ttn_list , ttno_list

# Example usage:
numbers = [3, 22, 84, 34, 90, 75, 87, 42, 70]  # Replace with your list of numbers
ttn_list , ttno_list = get_ttn_list(profiling_details, numbers)

In [ ]:
black_state = np.array([-1+2j , 1j])
white_state = np.array([1 , -2j])
ttn = ptn.alternating_product_state(ttn, black_state, white_state, bond_dim,  pattern = "checkerboard")
local_tensor = np.array([0 , 1])
ttn = ptn.uniform_product_state(ttn,local_state=local_tensor, bond_dim = bond_dim)

Lx = 3
Ly = 3
J_x =  1
J_y =  1 
J_z =  1 
h_z = 1
h_x = 0.5
H1 = ptn.Transverse_Field_Ising(J_z, h_x, Lx, Ly )
#H1 = ptn.Anisotropic_Heisenberg_ham(J_x, J_y, J_z, h_z, Lx, Ly , boundary_condition = "periodic")
H1 = H1.pad_with_identities(ttn , symbolic= True)
ttno = ptn.TTNO.from_hamiltonian(H1, ttn)

In [ ]:
from pytreenet.time_evolution.tdvp_algorithms.TJM import NoiseModel
# Define the noise model
gamma = 0.1
noise_model = NoiseModel(['relaxation', 'dephasing'], [gamma , gamma])
print(noise_model.processes)
print(noise_model.jump_operators)
print(noise_model.strengths)

In [ ]:
def generate_observable(Lx, Ly, matrix):
    observables = {}
    for i in range(Lx):
        for j in range(Ly):
            observables[f'Site({i},{j})'] = ptn.TensorProduct({f'Site({i},{j})': matrix})
    return observables

# Example usage
X , Y , Z  = ptn.pauli_matrices()
observables = generate_observable(Lx,Ly,X)

# observables["E"] = {"E": ttno}

# observable = dict{ value = site_id , key = tensor_product}
# observable = dict{ value = operator_name , key = TTNO}

# tdvp_tjm.results = dict{ key = site_id/operator_name , value = result for all times}
# np.array(list(tdvp_tjm.results.values())).shape = (#operators, results)


In [ ]:
expaction_params = {"mode" : "Krylov", # "Krylov" or "Taylor"
                    "size_threshold" : 200, # no higher approximation in default
                    # "Krylov" parameters
                    "Krylov_tol"  : 1e-5,
                    "krylov_dim"   : 6,
                    # "Taylor" parameters
                    "Taylor_num_terms"  : 3}

expansion_params = {"QR_Mode": ptn.SplitMode.KEEP,
                    "ExpansionMode": ptn.ExpansionMode.TTN,
                    "num_vecs": 3,
                    "tau": 0.01,
                    "SVDParameters": ptn.SVDParameters(max_bond_dim = np.inf , rel_tol= 1e-3 , total_tol = -np.inf),
                    "expansion_steps" : 10,
                    "tol": 0.2,
                    "tol_step_increase": 0.08,
                    "tol_step_decrease": 0.02,
                    "num_second_trial" : 10,
                    "max_bond": 40,
                    "rel_tot_bond": (1,10),}

config= ptn.TTNTimeEvolutionConfig(record_bond_dim = True,
                                   ExpAction_params= expaction_params,
                                   Expansion_params = expansion_params) 

ttn = normalize_ttn(ttn)
tdvp_tjm = GSE_TJM_TDVP_random(initial_state = ttn_list,
                    hamiltonian = ttno_list,
                    time_step_size = 0.1,
                    final_time = 5,
                    operators = observables,
                    N = 200,
                    noise_model = noise_model,
                    max_workers = 7,
                    config = config)

In [ ]:
import multiprocessing
import random
from copy import deepcopy

# 1) Initialize the arrays/dicts you need
tdvp_tjm.init_results_TJM(1)

normalized_states = [normalize_ttn(state) for state in tdvp_tjm.initial_state]

# 2) Create multiprocessing tools
manager = multiprocessing.Manager()
progress_queue = manager.Queue()
stop_event = manager.Event()

num_time_steps = len(tdvp_tjm.times())
total_steps = tdvp_tjm.N * num_time_steps


# Prepare argument tuples for each trajectory
args_list = []
for i in range(tdvp_tjm.N):
    # Choose a random index to pick a matching state and TTNo
    idx = random.randint(0, len(normalized_states) - 1)
    chosen_ttn = normalized_states[idx]
    chosen_hamiltonian = tdvp_tjm.hamiltonian[idx]

    args_list.append(
        (i, 1, chosen_ttn, chosen_hamiltonian, progress_queue)
    )

In [ ]:
args = args_list[0]

_ , evaluation_time,  initial_state, hamiltonian,  progress_queue = args
tdvp_tjm.state = deepcopy(initial_state)
tdvp_tjm.hamiltonian = hamiltonian

tdvp_tjm.update_path = tdvp_tjm._finds_update_path()
tdvp_tjm.orthogonalization_path = tdvp_tjm._find_tdvp_orthogonalization_path(tdvp_tjm.update_path)        
tdvp_tjm.backwards_update_path = tdvp_tjm._init_second_order_update_path()
tdvp_tjm.backwards_orth_path = tdvp_tjm._init_second_order_orth_path()

tol = tdvp_tjm.config.Expansion_params["tol"]

# save initial result
tdvp_tjm._orthogonalize_init()
tdvp_tjm.evaluate_and_save_results_TJM(evaluation_time, 0)

In [ ]:
stop

In [ ]:
tdvp_tjm.run_TJM(evaluation_time = 1 ,  filepath = r"C:\Users\edpou\Desktop\Ed\PyTreeNet-Project_10\PyTreeNet\TJM_results\mps200")
#tdvp_tjm.run_TJM(evaluation_time = 1 )

In [ ]:
tree_bonds = {0: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29],
 2: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 4: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 5: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 3: [24,
  24,
  24,
  24,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 1: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 6: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 7: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 8: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 10: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 12: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 11: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 9: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 13: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 15: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31],
 14: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 16: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31],
 18: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 17: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 19: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 20: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 21: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 22: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 23: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 24: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 25: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 26: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 27: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 29: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 28: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 30: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 31: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31],
 32: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 33: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 34: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 36: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 39: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29],
 35: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 37: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 38: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 41: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 40: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  25,
  25,
  25,
  25,
  25,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 42: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 43: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 44: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 45: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 46: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 47: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 48: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31],
 49: [24,
  24,
  24,
  24,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 50: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 51: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 52: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 53: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 54: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 55: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 56: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 57: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 58: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 59: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 60: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 61: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 62: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 63: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 65: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 64: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 66: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 67: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 68: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 69: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 71: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 70: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 72: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 73: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 74: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 75: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 76: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 77: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 78: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 79: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 81: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 80: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 82: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 83: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 84: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 86: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 88: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 87: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 85: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 89: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 90: [24,
  24,
  24,
  24,
  25,
  25,
  25,
  25,
  25,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 91: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 92: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 93: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 94: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 95: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 96: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 97: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 99: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 98: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  25,
  25,
  25,
  25,
  25,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 100: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 101: [24,
  24,
  24,
  24,
  24,
  24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 102: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 103: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 104: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 106: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 105: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 107: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 108: [24,
  24,
  24,
  24,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 109: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 110: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 111: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 112: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 113: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 114: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 115: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 117: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 116: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 118: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 119: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 120: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 121: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  24,
  24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 122: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 124: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 123: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 125: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 126: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 127: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 128: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 129: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 130: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 131: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 132: [24,
  24,
  24,
  24,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 134: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 133: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 136: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 135: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 137: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 138: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 139: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  25,
  25,
  25,
  25,
  25,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 141: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 140: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 142: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 143: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 144: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 145: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 148: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31],
 146: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 147: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 150: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 149: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 151: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 152: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 153: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 154: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 155: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 156: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 157: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 158: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 159: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 160: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 161: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 165: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29],
 162: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 163: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 164: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 166: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 167: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 168: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 170: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 169: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 172: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 171: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  25,
  25,
  25,
  25,
  25,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 173: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 174: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 175: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 177: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 176: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 179: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 178: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 180: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 181: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 182: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 183: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 184: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 186: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 185: [24,
  24,
  24,
  24,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 187: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 188: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 189: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 191: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 193: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31,
  31],
 190: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  28,
  28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 192: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 194: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 195: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 196: [24,
  24,
  24,
  24,
  28,
  28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 197: [24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 199: [24,
  24,
  24,
  24,
  26,
  26,
  26,
  26,
  26,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32],
 198: [24,
  24,
  24,
  24,
  27,
  27,
  27,
  27,
  27,
  29,
  29,
  29,
  29,
  29,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32]}

In [ ]:
mps_bonds = {5: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  38,
  38,
  38,
  38,
  38,
  41,
  41,
  41,
  41,
  41,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 1: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 4: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 3: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 0: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 2: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 6: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 7: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 8: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  38,
  38,
  38,
  38,
  38,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 12: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 10: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  36,
  36,
  36,
  36,
  36,
  42,
  42,
  42,
  42,
  42,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 9: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 13: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 11: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  47,
  47,
  47,
  47,
  47,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 18: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 14: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 15: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 16: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 17: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 19: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 20: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 21: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 24: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 22: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 23: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  46,
  46,
  46,
  46,
  46,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 25: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 27: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 26: [28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  38,
  38,
  38,
  38,
  38,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 33: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 28: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 29: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 31: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 32: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 30: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 34: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 35: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 37: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 36: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  41,
  41,
  41,
  41,
  41,
  47,
  47,
  47,
  47,
  47,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 40: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  39,
  39,
  39,
  39,
  39,
  41,
  41,
  41,
  41,
  41,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 38: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  44,
  44,
  44,
  44,
  44,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 39: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 41: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 42: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 44: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  37,
  37,
  37,
  37,
  37,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 43: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 46: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  42,
  42,
  42,
  42,
  42,
  43,
  43,
  43,
  43,
  43,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 45: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 47: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 48: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  38,
  38,
  38,
  38,
  38,
  41,
  41,
  41,
  41,
  41,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 49: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 50: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 53: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 52: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 51: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  47,
  47,
  47,
  47,
  47,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 55: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 54: [28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 59: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 56: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 57: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 58: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  46,
  46,
  46,
  46,
  46,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52],
 60: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  36,
  36,
  36,
  36,
  36,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 61: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 62: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  41,
  41,
  41,
  41,
  41,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 64: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 63: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 65: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 66: [28,
  28,
  28,
  28,
  30,
  30,
  30,
  30,
  30,
  39,
  39,
  39,
  39,
  39,
  47,
  47,
  47,
  47,
  47,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 68: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 69: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 67: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 70: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 71: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 72: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 74: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 73: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 76: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  38,
  38,
  38,
  38,
  38,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 75: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  43,
  43,
  43,
  43,
  43,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 78: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 77: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 79: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 80: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 81: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  46,
  46,
  46,
  46,
  46,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 82: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 83: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  41,
  41,
  41,
  41,
  41,
  49,
  49,
  49,
  49,
  49,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53,
  53],
 84: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 85: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 86: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  36,
  36,
  36,
  36,
  36,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 88: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 89: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 87: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 90: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 91: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 93: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 96: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 94: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 95: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 92: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 97: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 98: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 99: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 102: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 101: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 100: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 103: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  46,
  46,
  46,
  46,
  46,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 104: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  46,
  46,
  46,
  46,
  46,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 105: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 106: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43],
 107: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 108: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  38,
  38,
  38,
  38,
  38,
  46,
  46,
  46,
  46,
  46,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 109: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  36,
  36,
  36,
  36,
  36,
  39,
  39,
  39,
  39,
  39,
  42,
  42,
  42,
  42,
  42,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 110: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 111: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 112: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 113: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 114: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 115: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 116: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 117: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 118: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 119: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  46,
  46,
  46,
  46,
  46,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 120: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 121: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 123: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43],
 122: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  42,
  42,
  42,
  42,
  42,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 124: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 125: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 126: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 127: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 129: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 128: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  43,
  43,
  43,
  43,
  43,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 130: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 131: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 132: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 133: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  41,
  41,
  41,
  41,
  41,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 137: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 135: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 136: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 134: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 138: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  40,
  40,
  40,
  40,
  40,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 139: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  42,
  42,
  42,
  42,
  42,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 140: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  36,
  36,
  36,
  36,
  36,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 141: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 143: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  41,
  41,
  41,
  41,
  41,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43,
  43],
 144: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 142: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 145: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 146: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 148: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 150: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 147: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 149: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 151: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 152: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 153: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 154: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 155: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  42,
  42,
  42,
  42,
  42,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 156: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 157: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 158: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 159: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 160: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 162: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 161: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 163: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  37,
  37,
  37,
  37,
  37,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 164: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  44,
  44,
  44,
  44,
  44,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 165: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 166: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 167: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 168: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 169: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 170: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 172: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44,
  44],
 171: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  44,
  44,
  44,
  44,
  44,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 173: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 174: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 175: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  36,
  36,
  36,
  36,
  36,
  43,
  43,
  43,
  43,
  43,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 176: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 177: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 178: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  39,
  39,
  39,
  39,
  39,
  46,
  46,
  46,
  46,
  46,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51],
 179: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 180: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 181: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  42,
  42,
  42,
  42,
  42,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 182: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 183: [28,
  28,
  28,
  28,
  36,
  36,
  36,
  36,
  36,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 184: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  45,
  45,
  45,
  45,
  45,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 185: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  44,
  44,
  44,
  44,
  44,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 186: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48],
 187: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  44,
  44,
  44,
  44,
  44,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 188: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  42,
  42,
  42,
  42,
  42,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 189: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  40,
  40,
  40,
  40,
  40,
  45,
  45,
  45,
  45,
  45,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 191: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 190: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  46,
  46,
  46,
  46,
  46,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 192: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  38,
  38,
  38,
  38,
  38,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 194: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  43,
  43,
  43,
  43,
  43,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49,
  49],
 193: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  40,
  40,
  40,
  40,
  40,
  43,
  43,
  43,
  43,
  43,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46,
  46],
 195: [28,
  28,
  28,
  28,
  32,
  32,
  32,
  32,
  32,
  37,
  37,
  37,
  37,
  37,
  42,
  42,
  42,
  42,
  42,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45,
  45],
 196: [28,
  28,
  28,
  28,
  34,
  34,
  34,
  34,
  34,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 197: [28,
  28,
  28,
  28,
  33,
  33,
  33,
  33,
  33,
  41,
  41,
  41,
  41,
  41,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 199: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  40,
  40,
  40,
  40,
  40,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47,
  47],
 198: [28,
  28,
  28,
  28,
  35,
  35,
  35,
  35,
  35,
  41,
  41,
  41,
  41,
  41,
  45,
  45,
  45,
  45,
  45,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48,
  48]}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Convert TTN bond_dims dictionary to an array with shape (N, num_time_steps)
all_bond_dims_tree = np.array([np.array(tree_bonds[key]) for key in sorted(tree_bonds.keys())])
# Convert MPS bond_dims dictionary to an array with shape (N, num_time_steps)
all_bond_dims_mps = np.array([np.array(mps_bonds[key]) for key in sorted(mps_bonds.keys())])

# Compute the mean and standard deviation across trajectories for TTN
mean_bond_tree = np.mean(all_bond_dims_tree, axis=0)
std_bond_tree = np.std(all_bond_dims_tree, axis=0)

# Compute the mean and standard deviation across trajectories for MPS
mean_bond_mps = np.mean(all_bond_dims_mps, axis=0)
std_bond_mps = np.std(all_bond_dims_mps, axis=0)

plt.figure(figsize=(9, 5))
# Plot TTN results
plt.plot(times[0:99], mean_bond_tree, label='Spanning Tree', color='blue', linewidth=2)
plt.fill_between(times[0:99], mean_bond_tree - std_bond_tree, mean_bond_tree + std_bond_tree, color='blue', alpha=0.3, label='TTN Std. Dev.')
# Plot MPS results
plt.plot(times[0:99], mean_bond_mps, label='Snake-like mapping', color='red', linewidth=2)
plt.fill_between(times[0:99], mean_bond_mps - std_bond_mps, mean_bond_mps + std_bond_mps, color='red', alpha=0.3, label='MPS Std. Dev.')

plt.xlabel(r'$t\,J$', fontsize=10)
plt.ylabel(r'$Mean\ Total\ Bond\ \dim.\ \bar{\chi}(t)$', fontsize=10)
plt.legend(loc='best')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming tdvp_tjm.results is a dictionary with numerical arrays:
results = np.load(r"C:\Users\edpou\Desktop\Ed\PyTreeNet-Project_10\PyTreeNet\tjm.npz",  allow_pickle=True)
heatmap = np.array(list(results.values())[:-1])[:-1,:]
vmin = np.min(heatmap)
vmax = np.max(heatmap)

fig, ax = plt.subplots(1, 1, figsize=(8, 3))
L = len(tdvp_tjm.initial_state[0].nodes.keys())
im = ax.imshow(heatmap, aspect='auto', extent=[0, tdvp_tjm.final_time, L, 0],
                  cmap='viridis', vmin=vmin, vmax=vmax)
ax.set_ylabel('Site')

# Centers site ticks
ax.set_yticks(np.arange(0, L) + 0.5)
ax.set_yticklabels(np.arange(1, L+1))

fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming tdvp_tjm.results is a dictionary with numerical arrays:
heatmap = np.array(list(tdvp_tjm.results.values()))
vmin = np.min(heatmap)
vmax = np.max(heatmap)

fig, ax = plt.subplots(1, 1, figsize=(8, 3))
L = len(tdvp_tjm.initial_state[0].nodes.keys())
im = ax.imshow(heatmap, aspect='auto', extent=[0, tdvp_tjm.final_time, L, 0],
                  cmap='viridis', vmin=vmin, vmax=vmax)
ax.set_ylabel('Site')

# Centers site ticks
ax.set_yticks(np.arange(0, L) + 0.5)
ax.set_yticklabels(np.arange(1, L+1))

fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt


Lx, Ly = 3, 3
J = 1      # Ising coupling
g = 0.5    # Transverse field

tfi_system = ptn.TransverseFieldIsing2D_Qutip(Lx=Lx, Ly=Ly, J=J, g=g)

#-------------------------------
# Initial State
#-------------------------------
psi0 = tfi_system.uniform_product_state([0, 1])

#-------------------------------
# Build Collapse Operators
#-------------------------------
gamma_relax = 0.1
gamma_deph = 0.1
c_ops = tfi_system.build_c_ops(gamma_relax, gamma_deph)

#-------------------------------
# Observables
#-------------------------------
energy_op = tfi_system.H
#e_ops = tfi_system.sx_ops + [energy_op]
e_ops = tfi_system.sx_ops

#-------------------------------
# Evolve the System (mesolve)
#-------------------------------
end_time = 5
dt = 0.1
times = np.arange(0, end_time + dt, dt) 

result = qt.mesolve(
    H=tfi_system.H,
    rho0=psi0,
    tlist=times,
    c_ops=c_ops,
    e_ops=e_ops,
    progress_bar=True)

#-------------------------------
# Collect and Reshape the Data
#-------------------------------
data = np.array(result.expect)[:-1,:]
n_sites = tfi_system.n_sites  


n_sites = data.shape[0]  
n_times = data.shape[1]   
final_time = times[-1]       

# Determine color scale
vmin = np.min(data)
vmax = np.max(data)

# Create figure with 2 subplots
fig, ax = plt.subplots(1, 1, figsize=(8, 3))

# We only use ax[0] for the heatmap here, but you could put something else in ax[1]
im = ax.imshow(
    data, 
    aspect='auto',
    extent=[0, final_time, n_sites, 0],  # x: 0->final_time, y: top->bottom from n_sites->0
    cmap='viridis',
    vmin=vmin,
    vmax=vmax
)

# Label axes
ax.set_ylabel('Site')
ax.set_xlabel('Time')
ax.set_title(r'$ \langle \sigma_z \rangle $')

# Center the y-ticks so that each row is labeled with the site index
ax.set_yticks(np.arange(0, n_sites) + 0.5)  
ax.set_yticklabels(np.arange(1, n_sites + 1))

# Add a colorbar
cbar = fig.colorbar(im, ax=ax)
cbar.ax.set_title(r'$\langle \sigma_z \rangle$')


plt.tight_layout()
plt.show()

print("data.shape =", data.shape)

In [ ]:
energy_data = np.array(result.expect[-1]) - np.array(list(tdvp_tjm.results.values()))[-1,:] # Energy at each time step

plt.figure(figsize=(6, 4))
plt.plot(times, energy_data, label="Energy")
plt.xlabel("Time")
plt.ylabel("Energy")
plt.title("Energy vs Time")
plt.legend()
plt.show()

In [ ]:
# Calculate absolute error between the two energy computations:
error_data = np.abs(np.array(result.expect[-1]) - np.array(list(tdvp_tjm.results.values()))[-1, :])

plt.figure(figsize=(6, 4))
plt.semilogy(times, error_data, label="Energy Error")
plt.xlabel("Time")
plt.ylabel("Absolute Error (log scale)")
plt.title("Energy Error vs Time (Semilog)")
plt.legend()
plt.show()

In [ ]:
integrated_error = np.trapz(np.abs(energy_data), times)
mean_error = np.mean(np.abs(energy_data))
max_error = np.max(np.abs(energy_data))
print(f"Integrated Error: {integrated_error}")
print(f"Mean Error: {mean_error}")
print(f"Max Error: {max_error}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Suppose:
#   exact_result = sz_data          # shape (9, n_times)
#   tdvp_result  = tdvp.results[:-1,:]  # also shape (9, n_times)
# (The "[:-1, :]" is removing any extra row if your data had time in the last row.)

# 1) Compute the difference
exact_result = data
tdvp_result = np.array(list(tdvp_tjm.results.values()))[:-1,:]
error_data = exact_result - tdvp_result  # shape (n_sites=9, n_times)
error_data = np.real(error_data)

# 2) Build a time array if needed:
n_sites, n_times = error_data.shape
final_time = 10.0     # or however you define it
dt = final_time / (n_times - 1)
time_values = np.linspace(0, final_time, n_times)

# 3) Plot the heatmap of the error
fig, ax = plt.subplots(figsize=(8, 3))

im = ax.imshow(
    error_data,
    aspect='auto',
    origin='upper',                # row=0 at the top
    extent=[time_values[0], time_values[-1], 0, n_sites],
    cmap='RdBu',                   # diverging colormap to highlight positive/negative
    vmin=error_data.min(),
    vmax=error_data.max()
)

ax.set_xlabel("Time")
ax.set_ylabel("Site Index")
ax.set_title("Error in ⟨σ_z⟩ (Exact - Tensor Network)")

# Invert y-axis if you want site 0 at the bottom
ax.invert_yaxis()

# Center tick labels, e.g. for site indices
ax.set_yticks(np.arange(n_sites) + 0.5)
ax.set_yticklabels(np.arange(1, n_sites + 1))

# Add colorbar
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Difference in ⟨σ_z⟩")

plt.tight_layout()
plt.show()


In [ ]:
error_data.shape
row_sums = np.sum(error_data, axis=0)

plt.plot(row_sums)
plt.xlabel('Time')
plt.ylabel('Sum of Errors')
plt.title('Sum of Errors Over Time')
plt.show()

In [ ]:
mps_results  = np.load(r"C:\Users\edpou\Desktop\Ed\PyTreeNet-Project_10\PyTreeNet\TJM_results\mps200.npz",  allow_pickle=True)
mps_results  = np.array(list(mps_results.values())[:-1])
tree_results = np.load(r"C:\Users\edpou\Desktop\Ed\PyTreeNet-Project_10\PyTreeNet\TJM_results\tree200.npz",  allow_pickle=True)
tree_results = np.array(list(tree_results.values())[:-1])
exact_result = np.array(result.expect)
times = tdvp_tjm.times()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming tdvp_tjm.results is a dictionary with numerical arrays:
heatmap = tdvp_tjm.results[:,:] - exact_result[:,:]
vmin = np.min(heatmap)
vmax = np.max(heatmap)

fig, ax = plt.subplots(1, 1, figsize=(8, 3))
L = len(tdvp_tjm.state.nodes.keys())
im = ax.imshow(heatmap, aspect='auto', extent=[0, tdvp_tjm.final_time, L, 0],
                  cmap='viridis', vmin=vmin, vmax=vmax)
ax.set_ylabel('Site')

# Centers site ticks
ax.set_yticks(np.arange(0, L) + 0.5)
ax.set_yticklabels(np.arange(1, L+1))

fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Compute per-site errors
error_per_site_mps = np.abs(mps_results[0:-1, :] - exact_result[0:-1, :])
total_error_mps    = np.sum(error_per_site_mps, axis=0)

error_per_site_ttn = np.abs(tree_results[0:-1, :] - exact_result[0:-1, :])
total_error_ttn    = np.sum(error_per_site_ttn, axis=0)

# Compute the accumulated error (cumulative sum)
acc_error_mps = np.cumsum(total_error_mps)
acc_error_ttn = np.cumsum(total_error_ttn)

# Plot cumulative errors
plt.figure(figsize=(9, 3))
plt.plot(times, acc_error_mps, label='Snake-like mapping' ,color='red' )
plt.plot(times, acc_error_ttn, label='Spanning Tree' , color='blue')
plt.xlabel(r'$t\,J$', fontsize=10)
plt.ylabel('TAE', fontsize=10)
plt.legend()
plt.grid(True, alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
np.sum(total_error_mps) , np.sum(total_error_ttn)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Compute the absolute error for each method
error_mps = np.abs(mps_results[-1, :] - exact_result[-1, :])
error_tree = np.abs(tree_results[-1, :] - exact_result[-1, :])

# Create the plot
plt.figure(figsize=(12, 3))
plt.plot(times, error_mps, label='Snake-like mapping', marker='o', linestyle='-', linewidth=1, markersize=3)
plt.plot(times, error_tree, label='Spanning Tree', marker='s', linestyle='-', linewidth=1, markersize=3)
plt.xlabel(r'$t\,J$', fontsize=10)
plt.ylabel(r'$\left|\langle \hat{H} \rangle_{\mathrm{approx}} - \langle \hat{H} \rangle_{\mathrm{exact}}\right|$', fontsize=10)
plt.legend(fontsize=10)
plt.grid(True)
plt.show()


In [ ]:
np.sum(error_mps) , np.sum(error_tree)